# 🤖 Cyber-Enhanced AI Generator for Brand Voice on Social Media

This Colab notebook builds a complete **AI brand content generator** that can mimic the voice of top global brands such as Nike, Apple, Tesla, Coca-Cola, and more.

### What this app does:
• Generates **on-brand, platform-specific content** (Twitter, Instagram, LinkedIn, Facebook)  
• Produces **ads, captions, product descriptions, blog intros**  
• Supports **brand personality modeling**  
• Gives **clean, formatted output** with hashtags highlighted  

---

## 📘 How This Notebook Works

1. Install all dependencies  
2. Login to Hugging Face  
3. Write Flask backend (`app.py`)  
4. Write HTML frontend  
5. Write CSS styling  
6. Run Flask and open ngrok link  

You DO NOT need any dataset — only user input.

---

## 🔥 Why this project?

Marketing teams spend hours writing platform-specific content.  
This app generates **consistent, polished, brand-aligned content instantly**.

Perfect for:
- Social media agencies  
- Marketing students  
- Content creators  
- Automation workflows  


In [ ]:
!pip install flask pyngrok transformers accelerate bitsandbytes
!mkdir -p templates static uploads

# 🔐 Hugging Face Authentication — Required Step

This project uses the model:

mosaicml/mpt-7b-instruct  

This model requires access through Hugging Face.

### Steps:
1. Visit https://huggingface.co/settings/tokens  
2. Click "New Token" → Select **Read** permission  
3. Copy the token  
4. Paste it inside login(token="YOUR_TOKEN")

This allows the notebook to download and run the AI model.

⚠️ Do NOT share your Hugging Face token publicly.


In [ ]:
from huggingface_hub import login
login(token="🔑 replace with your Hugging Face token")  # 🔑 replace with your Hugging Face token

# 🧠 Flask Backend — Brand Voice AI Engine

This cell creates the complete Flask backend.

### Contains:

### 1️⃣ Predefined Brand Voice Profiles
Nike → Bold & Motivational  
Apple → Minimal, premium  
Coca-Cola → Joyful & universal  
Tesla → Futuristic & disruptive  
… and many more.

If the user enters ANY unknown brand, the app uses a **default friendly tone**.

---

### 2️⃣ Lazy-Loaded AI Model (7B parameters)

The project uses:

• mosaicml/mpt-7b-instruct  
• Offloading enabled (saves GPU memory)  
• `@lru_cache` ensures model loads only once  

While loading, UI shows:

⚡ Model is downloading or initializing. Please wait...

---

### 3️⃣ Prompt Generator

`create_prompt()` produces high-quality prompts combining:

• Brand voice  
• Tone  
• Social media platform rules  
• Content type  
• Topic  

---

### 4️⃣ Output Formatting

Mentions (@brand) → bold  
Hashtags (#trend) → highlighted  
Breaks text into clean readable lines  

---

### 5️⃣ Routes

"/" → Web UI  
POST → Generate content  
Returns formatted lines into the frontend  

You generally do NOT modify this backend unless adding new AI features.


In [ ]:
%%writefile app.py
from flask import Flask, render_template, request
import functools, re, torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import os

app = Flask(__name__)

# ===============================
# 🔹 Brand Voice Profiles
# ===============================
BRAND_STYLES = {
    "Nike": "Motivational, bold, athletic, inspiring. Punchy lines. Example: 'Just Do It'.",
    "Apple": "Minimalist, sleek, premium, aspirational. Innovation + design. Example: 'Think Different'.",
    "Starbucks": "Warm, community-driven, friendly. Coffee culture, cozy seasonal tones.",
    "Adidas": "Performance-driven, energetic, youthful. Focus on athletes + innovation.",
    "Coca-Cola": "Joyful, celebratory, universal. Happiness, sharing, togetherness.",
    "Tesla": "Futuristic, bold, disruptive. Sustainability + cutting-edge tech.",
    "Disney": "Magical, family-friendly, imaginative. Wonder, storytelling, dreams.",
    "Amazon": "Customer-first, practical, innovative. Convenience + reliability.",
    "Google": "Helpful, innovative, empowering. Tech + AI made simple.",
    "Samsung": "Stylish, global, premium. Next-gen innovation with sleek design."
}

DEFAULT_STYLE = "Friendly, engaging, persuasive, and approachable. Short catchy lines with hashtags."

# ===============================
# 🔹 Lazy-load Model with offloading
# ===============================
MODEL_LOADING = False  # Flag to show loading status

@functools.lru_cache(maxsize=1)
def get_generator():
    global MODEL_LOADING
    MODEL_LOADING = True

    MODEL_NAME = "mosaicml/mpt-7b-instruct"
    offload_folder = "offload"
    os.makedirs(offload_folder, exist_ok=True)

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        offload_folder=offload_folder
    )

    generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=150,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True
    )

    MODEL_LOADING = False
    return generator

# ===============================
# 🔹 Format Posts for HTML
# ===============================
def format_post(text):
    text = re.sub(r'(@\w+)', r'<strong>\1</strong>', text)
    text = re.sub(r'(#\w+)', r'<em>\1</em>', text)
    return [line.strip() for line in text.split("\n") if line.strip()]

# ===============================
# 🔹 Generate Content-Focused Prompt
# ===============================
def create_prompt(brand_name, tone, platform, topic, content_type, style):
    """Creates a completion-style prompt that leads directly to content generation"""

    # Platform-specific guidelines
    platform_guide = {
        "Twitter": "Keep it under 280 characters. Use 2-3 hashtags. Be punchy and engaging.",
        "Instagram": "Visual and engaging. Use emojis and 5-8 hashtags. Tell a story.",
        "LinkedIn": "Professional and informative. Focus on value and insights. 2-3 hashtags max.",
        "Facebook": "Conversational and relatable. Encourage engagement. Use 1-3 hashtags."
    }

    guide = platform_guide.get(platform, "Be engaging and on-brand.")

    # Create a direct completion prompt
    prompt = f"""Brand: {brand_name}
Platform: {platform}
Topic: {topic}
Tone: {tone}
Style: {style}
Guidelines: {guide}

{content_type} for {platform}:
"""

    return prompt

# ===============================
# 🔹 Routes
# ===============================
@app.route("/", methods=["GET", "POST"])
def home():
    posts = []
    model_loading = MODEL_LOADING
    if request.method == "POST":
        brand_name = request.form.get("brand_name").strip()
        tone = request.form.get("tone").strip()
        platform = request.form.get("platform").strip()
        topic = request.form.get("topic").strip()
        content_type = request.form.get("content_type").strip()

        style = BRAND_STYLES.get(brand_name, DEFAULT_STYLE)

        # Create the prompt
        prompt = create_prompt(brand_name, tone, platform, topic, content_type, style)

        try:
            generator = get_generator()
            output = generator(prompt)[0]["generated_text"]

            # Extract only the generated content (remove the prompt)
            generated_content = output.replace(prompt, "").strip()

            # Clean up any meta-instructions that might appear
            generated_content = re.sub(r'(Here\'s|Here is|I\'ll create|Let me create).*?:', '', generated_content, flags=re.IGNORECASE)
            generated_content = re.sub(r'^(Sure|Okay|Alright)[,!.]?\s*', '', generated_content, flags=re.IGNORECASE)

            posts = format_post(generated_content)

            # If posts are empty or too short, provide fallback
            if not posts or (len(posts) == 1 and len(posts[0]) < 10):
                posts = [f"⚠️ Generated content was too short. Try adjusting your inputs or try again."]

        except Exception as e:
            posts = [f"❌ Error: {e}"]

    return render_template("index.html", posts=posts, model_loading=MODEL_LOADING)

# ===============================
# 🔹 Run
# ===============================
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000, debug=False)


# 🌐 Frontend UI — Styled Brand Voice Content Generator

This HTML file provides:

✔ Clean input form  
✔ Fields for brand, tone, platform, topic, content type  
✔ Real-time model loading notice  
✔ Result box for generated content  
✔ "Copy to Clipboard" button  
✔ Auto-cleaned styling for mentions + hashtags  

The user interacts ONLY with this interface.  
All AI generation happens through Flask in the backend.


In [ ]:
%%writefile templates/index.html

<!DOCTYPE html>
<html>
<head>
    <title>🤖 Brand Voice AI Generator</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="container">
        <h1>🤖 AI Brand Voice Content Generator</h1>
        <p class="subtitle">Generate authentic brand content in seconds</p>

        {% if model_loading %}
        <div class="loading">
            ⚡ Model is downloading or initializing. Please wait...
        </div>
        {% endif %}

        <div class="card">
            <form method="post">
                <div class="form-group">
                    <label>🏢 Brand Name</label>
                    <input type="text" name="brand_name" placeholder="e.g., Nike, Apple, Starbucks" required>
                </div>

                <div class="form-group">
                    <label>🎨 Tone / Style</label>
                    <input type="text" name="tone" placeholder="e.g., Friendly, Bold, Professional, Witty" required>
                </div>

                <div class="form-group">
                    <label>📱 Platform</label>
                    <select name="platform" required>
                        <option value="">Select Platform</option>
                        <option value="Twitter">Twitter</option>
                        <option value="Instagram">Instagram</option>
                        <option value="LinkedIn">LinkedIn</option>
                        <option value="Facebook">Facebook</option>
                    </select>
                </div>

                <div class="form-group">
                    <label>💡 Topic / Campaign Theme</label>
                    <input type="text" name="topic" placeholder="e.g., New product launch, Summer sale, Motivational Monday" required>
                </div>

                <div class="form-group">
                    <label>📝 Content Type</label>
                    <select name="content_type" required>
                        <option value="">Select Content Type</option>
                        <option value="Social Post">Social Post</option>
                        <option value="Ad Copy">Ad Copy</option>
                        <option value="Promotional Flyer">Promotional Flyer</option>
                        <option value="Blog Intro">Blog Intro</option>
                        <option value="Product Description">Product Description</option>
                    </select>
                </div>

                <button type="submit">✨ Generate Content</button>
            </form>
        </div>

        {% if posts %}
        <div class="result">
            <h2>✅ Generated Content</h2>
            <div class="content-box">
                {% for post in posts %}
                <p class="post-line">{{ post | safe }}</p>
                {% endfor %}
            </div>
            <button class="copy-btn" onclick="copyContent()">📋 Copy Content</button>
        </div>
        {% endif %}
    </div>

    <script>
        function copyContent() {
            const contentBox = document.querySelector('.content-box');
            const text = contentBox.innerText;
            navigator.clipboard.writeText(text).then(() => {
                const btn = document.querySelector('.copy-btn');
                btn.textContent = '✅ Copied!';
                setTimeout(() => {
                    btn.textContent = '📋 Copy Content';
                }, 2000);
            });
        }
    </script>
</body>
</html>


# 🎨 CSS Styling — Gradient UI + Smooth Animations

This stylesheet includes:

• Dark cyber-themed gradient background  
• Gold accent highlights (for premium brand feel)  
• Smooth animations (slide-in)  
• Highlighted content box  
• Responsive design  
• Copy button styling  
• Error/loading states  

You can replace the color palette to match your brand or theme.

This file controls the look-and-feel of the entire UI.


In [ ]:
%%writefile static/style.css

* {
    margin: 0;
    padding: 0;
    box-sizing: border-box;
}

body {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    background: linear-gradient(135deg, #0f2027, #203a43, #2c5364);
    color: white;
    display: flex;
    justify-content: center;
    align-items: flex-start;
    padding: 30px 20px;
    min-height: 100vh;
}

.container {
    text-align: center;
    width: 100%;
    max-width: 700px;
}

h1 {
    margin-bottom: 10px;
    font-size: 2.2em;
    font-weight: 700;
    text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.3);
}

.subtitle {
    font-size: 1.1em;
    opacity: 0.9;
    margin-bottom: 30px;
    color: #FFD700;
}

.card {
    background: rgba(255, 255, 255, 0.1);
    backdrop-filter: blur(10px);
    padding: 30px;
    border-radius: 15px;
    margin-bottom: 25px;
    box-shadow: 0 8px 32px rgba(0, 0, 0, 0.3);
}

.form-group {
    margin-bottom: 20px;
    text-align: left;
}

.form-group label {
    display: block;
    margin-bottom: 8px;
    font-weight: 600;
    font-size: 0.95em;
    color: #FFD700;
}

form input,
form select {
    width: 100%;
    padding: 12px 15px;
    border-radius: 8px;
    border: 2px solid rgba(255, 255, 255, 0.2);
    font-size: 16px;
    background: rgba(255, 255, 255, 0.9);
    color: #333;
    transition: all 0.3s ease;
}

form input:focus,
form select:focus {
    outline: none;
    border-color: #FFD700;
    background: white;
    box-shadow: 0 0 0 3px rgba(255, 215, 0, 0.1);
}

form input::placeholder {
    color: #999;
}

button {
    width: 100%;
    padding: 14px;
    background: linear-gradient(135deg, #FFD700, #FFA500);
    color: #000;
    border: none;
    border-radius: 8px;
    font-size: 18px;
    font-weight: bold;
    cursor: pointer;
    transition: all 0.3s ease;
    margin-top: 10px;
    text-transform: uppercase;
    letter-spacing: 1px;
}

button:hover {
    transform: translateY(-2px);
    box-shadow: 0 6px 20px rgba(255, 215, 0, 0.4);
}

button:active {
    transform: translateY(0);
}

.result {
    background: rgba(255, 255, 255, 0.1);
    backdrop-filter: blur(10px);
    padding: 30px;
    border-radius: 15px;
    box-shadow: 0 8px 32px rgba(0, 0, 0, 0.3);
    animation: slideIn 0.5s ease;
}

@keyframes slideIn {
    from {
        opacity: 0;
        transform: translateY(20px);
    }
    to {
        opacity: 1;
        transform: translateY(0);
    }
}

.result h2 {
    margin-bottom: 20px;
    color: #FFD700;
    font-size: 1.8em;
}

.content-box {
    background: rgba(0, 0, 0, 0.3);
    padding: 20px;
    border-radius: 10px;
    margin-bottom: 15px;
    text-align: left;
    border-left: 4px solid #FFD700;
    min-height: 100px;
}

.post-line {
    margin: 10px 0;
    line-height: 1.6;
    font-size: 1.05em;
}

.copy-btn {
    background: rgba(255, 255, 255, 0.2);
    color: white;
    width: auto;
    padding: 10px 25px;
    font-size: 14px;
    margin-top: 10px;
    border: 2px solid rgba(255, 255, 255, 0.3);
}

.copy-btn:hover {
    background: rgba(255, 255, 255, 0.3);
    border-color: #FFD700;
}

strong {
    color: #FFD700;
    font-weight: 700;
}

em {
    color: #87CEEB;
    font-style: normal;
    font-weight: 600;
}

.loading {
    background: rgba(255, 165, 0, 0.2);
    border: 2px solid #FFA500;
    padding: 15px;
    border-radius: 10px;
    margin-bottom: 20px;
    font-weight: bold;
    animation: pulse 2s infinite;
}

@keyframes pulse {
    0%, 100% {
        opacity: 1;
    }
    50% {
        opacity: 0.7;
    }
}

/* Responsive Design */
@media (max-width: 768px) {
    h1 {
        font-size: 1.8em;
    }

    .card, .result {
        padding: 20px;
    }

    .subtitle {
        font-size: 1em;
    }
}


📘 Kill Previous Processes

This ensures Flask and ngrok do not conflict:

- Stops earlier Flask sessions  
- Stops older ngrok tunnels  
- Prevents "port already in use" errors  

Safe to run every time before starting server.


In [ ]:
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"


📘  Checking Port 8000 (User Instructions)

If server fails, port 8000 may be occupied.

Run:
!lsof -i :8000

If you see:
python   12345 LISTEN

Kill it with:
!kill -9 12345

Then launch Flask again.


In [ ]:
!lsof -i :8000

In [ ]:
!kill -9 25018  # here ID no shud be changed based on the above cell { PID }

📘  Run Flask App in Background

Starts backend without blocking the notebook:

!nohup python app.py > flask.log 2>&1 &

Logs are stored in flask.log


In [ ]:
!nohup python app.py > flask.log 2>&1 &

📘  Ngrok Setup

Ngrok provides a public HTTPS link.

Your ngrok token was removed for safety.

To use ngrok:
1. Get token → https://dashboard.ngrok.com/get-started/your-authtoken  
2. Add inside notebook:

conf.get_default().auth_token = "YOUR_NGROK_TOKEN_HERE"

3. Start tunnel:

public_url = ngrok.connect(8000)

Shareable app link appears here.


In [ ]:
# ===============================
# 8️⃣ Start ngrok tunnel
# ===============================
from pyngrok import ngrok, conf
conf.get_default().auth_token = "YOUR_NGROK_TOKEN_HERE"  # 🔑 replace with your token

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

# ===============================
# 9️⃣ Check logs (optional)
# ===============================
!sleep 3 && tail -n 20 flask.log


📘  View Logs

To debug backend:

!tail -n 20 flask.log

Shows:
- Model loading issues  
- Prompt errors  
- Script formatting errors  
- Runtime crashes  


In [ ]:
!tail -n 50 flask.log